## 0. Cuda Test

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

%pip install numpy

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

import numpy
print(numpy.__version__)

2.7.0+cu118
True
2.3.1


## 1. Setup

### 1.1 Install packages

In [3]:
%pip install git+https://github.com/brandonstarxel/chunking_evaluation.git --quiet
%pip install hf_xet --quiet
%pip install bitsandbytes --quiet
%pip install --upgrade transformers --quiet
%pip install --upgrade chromadb --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### 1.2 Import Required Modules

In [4]:
import os
import sys

# 取得目前筆記本檔案的絕對路徑
notebook_path = os.path.abspath('chunking_exp.ipynb')

# 取得筆記本所在的目錄（即專案根目錄）
project_root = os.path.dirname(notebook_path)

# 將專案根目錄（包含你的本地 chunking_evaluation 資料夾）
# 插入到 sys.path 的最前面，確保優先載入本地版本
sys.path.insert(0, project_root)
print(f"已將專案根目錄 '{project_root}' 優先加入到 Python 模組搜尋路徑。")


from chunking_evaluation.chunking import FixedTokenChunker, RecursiveTokenChunker
from chunking_evaluation import GeneralEvaluation, SyntheticEvaluation
from chunking_evaluation.utils import bge_m3_token_count, get_bge_m3_embedding_function
from chunking_evaluation.evaluation_framework.general_evaluation_data.DatasetAnalyzer import DatasetAnalyzer
import pandas as pd
from IPython.display import display, clear_output
import http.client
import json


已將專案根目錄 'd:\ArtificialIntelligenceCustomerService\code\exp' 優先加入到 Python 模組搜尋路徑。


## 2. Create Chunkers

### 2.1 RecursiveTokenChunker & FixedTokenChunker

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = [512, 1024, 2048, 4096]
chunk_overlap = [50, 100, 150, 200, 250]

chunkers = [
    RecursiveCharacterTextSplitter(
        chunk_size=cs,
        chunk_overlap=co,
        length_function=len,
        separators=["\n\n", "\n", " ", "", "。", "！", "？", "，", "."]
    )
    for cs in chunk_size
    for co in chunk_overlap
]

# 範例：印出每個 chunker 的設定
for i, chunker in enumerate(chunkers):
    print(f"Chunker {i+1}: chunk_size={chunker._chunk_size}, chunk_overlap={chunker._chunk_overlap}")

Chunker 1: chunk_size=512, chunk_overlap=50
Chunker 2: chunk_size=512, chunk_overlap=100
Chunker 3: chunk_size=512, chunk_overlap=150
Chunker 4: chunk_size=512, chunk_overlap=200
Chunker 5: chunk_size=512, chunk_overlap=250
Chunker 6: chunk_size=1024, chunk_overlap=50
Chunker 7: chunk_size=1024, chunk_overlap=100
Chunker 8: chunk_size=1024, chunk_overlap=150
Chunker 9: chunk_size=1024, chunk_overlap=200
Chunker 10: chunk_size=1024, chunk_overlap=250
Chunker 11: chunk_size=2048, chunk_overlap=50
Chunker 12: chunk_size=2048, chunk_overlap=100
Chunker 13: chunk_size=2048, chunk_overlap=150
Chunker 14: chunk_size=2048, chunk_overlap=200
Chunker 15: chunk_size=2048, chunk_overlap=250
Chunker 16: chunk_size=4096, chunk_overlap=50
Chunker 17: chunk_size=4096, chunk_overlap=100
Chunker 18: chunk_size=4096, chunk_overlap=150
Chunker 19: chunk_size=4096, chunk_overlap=200
Chunker 20: chunk_size=4096, chunk_overlap=250


## 3. Prepare Corpora

In [6]:
'''
def download_text(book_id, file_name, directory):
    conn = http.client.HTTPSConnection("www.gutenberg.org")
    url = f"/files/{book_id}/{book_id}-0.txt"

    conn.request("GET", url)
    response = conn.getresponse()

    if response.status == 200:
        text = response.read().decode('utf-8')

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the text to the specified file within the directory
        file_path = os.path.join(directory, file_name)
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text)
        print(f"Book '{file_name}' downloaded and saved successfully in '{directory}'.")
    else:
        print(f"Failed to download the book. Status code: {response.status}")


# Define the directory to save the books
directory = "corpora"

def download_example_texts(download=False, directory="./corpora"):
    if download:
        books = {
            1661: "the_adventures_of_sherlock_holmes.txt",
            1342: "pride_and_prejudice.txt", 
            174: "the_picture_of_dorian_gray.txt"
        }
        print("開始下載範例文本...")

        for book_id, file_name in books.items():
            try:
                download_text(book_id, file_name, directory)
                print(f"✅ 成功下載: {file_name}")
            except Exception as e:
                print(f"❌ 下載失敗 {file_name}: {e}")
        
        print("範例文本下載完成！")
    else:
        print("跳過下載範例文本。")

download_example_texts(download=False)
'''

'\ndef download_text(book_id, file_name, directory):\n    conn = http.client.HTTPSConnection("www.gutenberg.org")\n    url = f"/files/{book_id}/{book_id}-0.txt"\n\n    conn.request("GET", url)\n    response = conn.getresponse()\n\n    if response.status == 200:\n        text = response.read().decode(\'utf-8\')\n\n        # Create directory if it does not exist\n        os.makedirs(directory, exist_ok=True)\n\n        # Save the text to the specified file within the directory\n        file_path = os.path.join(directory, file_name)\n        with open(file_path, "w", encoding="utf-8") as file:\n            file.write(text)\n        print(f"Book \'{file_name}\' downloaded and saved successfully in \'{directory}\'.")\n    else:\n        print(f"Failed to download the book. Status code: {response.status}")\n\n\n# Define the directory to save the books\ndirectory = "corpora"\n\ndef download_example_texts(download=False, directory="./corpora"):\n    if download:\n        books = {\n           

## 4. Initialize the Evaluation Environment

### 4.1 Setup Path

In [7]:
import os
import glob
import pandas as pd
from typing import List, Dict

# 確保你的 chunking_evaluation 模組已經載入
from chunking_evaluation import GeneralEvaluation, SyntheticEvaluation
from chunking_evaluation.evaluation_framework.general_evaluation_data.DatasetAnalyzer import DatasetAnalyzer

# 設定路徑
corpora_directory = "./chunking_evaluation/evaluation_framework/general_evaluation_data/corpora"
queries_csv_path = "./chunking_evaluation/evaluation_framework/general_evaluation_data/generated_queries_and_excerpts.csv"


### 4.2 Already have corpora & queries csv

In [8]:

# 檢查 CSV 檔案是否存在
if os.path.exists(queries_csv_path):
    print(f"✅ 偵測到評估資料集 CSV 檔案 '{queries_csv_path}' 已存在。")
    print("將使用 GeneralEvaluation 進行評估。")
    evaluation = GeneralEvaluation()
else:
    print(f"🚧 未偵測到評估資料集 CSV 檔案，將使用 SyntheticEvaluation 進行生成。")

✅ 偵測到評估資料集 CSV 檔案 './chunking_evaluation/evaluation_framework/general_evaluation_data/generated_queries_and_excerpts.csv' 已存在。
將使用 GeneralEvaluation 進行評估。


### 4.3 Generate queries if haven't

In [9]:
%pip install --upgrade ipywidgets --quiet
%pip install accelerate --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:

# 檢查 CSV 檔案是否存在
if os.path.exists(queries_csv_path):
    print(f"✅ 偵測到評估資料集 CSV 檔案 '{queries_csv_path}' 已存在。")
    print("將使用 GeneralEvaluation 進行評估。")
else:
    print(f"🚧 未偵測到評估資料集 CSV 檔案，將使用 SyntheticEvaluation 進行生成。")

    # 進行資料取樣以生成評估資料集
    analyzer = DatasetAnalyzer(corpora_directory)
    analyzer.analyze_folder()
    analyzer.generate_report()
    sample = analyzer.get_stratified_sample(100)
    
    # 建立檔案路徑清單
    corpora_paths_list = [os.path.normpath(file_info['file_path']).replace('\\\\', '/') for file_info in sample]
    corpora_id_paths = {
        os.path.basename(path): path
        for path in corpora_paths_list
    }

    print(f"\n🎯 分層取樣完成，選擇了 {len(sample)} 個檔案。")
    print("將使用 SyntheticEvaluation 生成評估資料集並進行評估。")

    # 初始化 SyntheticEvaluation
    evaluation = SyntheticEvaluation(corpora_id_paths, queries_csv_path)

    # 生成評估資料集
    print(f"🚀 正在生成查詢與摘錄...")
    evaluation.generate_queries_and_excerpts(approximate_excerpts=True, num_rounds=1, queries_per_corpus=1)
    print(f"✔️ 查詢生成完成，已儲存至 '{queries_csv_path}'")
    #evaluation.repair_csv_references()
    #evaluation.validate_csv_integrity()

✅ 偵測到評估資料集 CSV 檔案 './chunking_evaluation/evaluation_framework/general_evaluation_data/generated_queries_and_excerpts.csv' 已存在。
將使用 GeneralEvaluation 進行評估。


In [11]:
# Apply filter to remove queries with poor excerpts
#evaluation.filter_poor_excerpts(threshold=0.36)

# Apply filter to remove duplicates
#evaluation.filter_duplicates(threshold=0.6)

### 4.4 Finished Initialization

In [12]:
print("\n🎉 評估環境初始化完成。")


🎉 評估環境初始化完成。


## 5. Run the evaluation

In [13]:
import torch
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 是否可用: {torch.cuda.is_available()}")
print(f"CUDA 裝置數量: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"當前使用的 CUDA 裝置: {torch.cuda.current_device()}")
    print(f"CUDA 裝置名稱: {torch.cuda.get_device_name(0)}")

PyTorch 版本: 2.7.0+cu118
CUDA 是否可用: True
CUDA 裝置數量: 1
當前使用的 CUDA 裝置: 0
CUDA 裝置名稱: NVIDIA GeForce RTX 4070 Ti SUPER


In [14]:
import shutil
import pandas as pd
from IPython.display import display

# 刪除舊的 Chroma DB，以確保每次評估都是從頭開始
chroma_db_path = "./chunking_evaluation/evaluation_framework/general_evaluation_data/questions_db"
if os.path.exists(chroma_db_path):
    print(f"🔄 正在刪除舊的 Chroma DB: {chroma_db_path}")
    shutil.rmtree(chroma_db_path)

# ====== 將載入嵌入函式的程式碼移到迴圈外面 ======
print("🚀 準備載入嵌入模型，此過程僅執行一次...")
embedding_function = get_bge_m3_embedding_function()
print("✅ 嵌入模型載入完成！")
print("--------------------------------------------------")
# ======================================================

results = []

for chunker in chunkers:
    print(f"⚙️ 正在使用 {chunker.__class__.__name__} 進行評估...")

    # 執行評估時，直接使用已經載入好的 embedding_function
    result = evaluation.run(
        chunker,
        embedding_function,  # 這裡使用已經載入好的函式
        retrieve=5,
        db_to_save_chunks=chroma_db_path
    )

    # 清理結果並新增 chunker 資訊
    chunk_size = chunker._chunk_size if hasattr(chunker, '_chunk_size') else 0
    chunk_overlap = chunker._chunk_overlap if hasattr(chunker, '_chunk_overlap') else 0
    result['chunker'] = chunker.__class__.__name__ + f"_{chunk_size}_{chunk_overlap}"

    results.append(result)

    # 在每次迭代後，顯示當前 chunker 的結果
    current_df = pd.DataFrame([result])
    print("\n--- 當前 Chunking 策略結果 ---")
    display(current_df)
    print("----------------------------\n")

print("\n✅ 所有分塊器評估完成！")

# 顯示包含所有結果的總結 DataFrame
final_df = pd.DataFrame(results)
print("=== 最終總結報告 ===")
display(final_df)

🔄 正在刪除舊的 Chroma DB: ./chunking_evaluation/evaluation_framework/general_evaluation_data/questions_db
🚀 準備載入嵌入模型，此過程僅執行一次...
正在載入 BGE-M3 模型到裝置: cuda
✅ 嵌入模型載入完成！
--------------------------------------------------
⚙️ 正在使用 RecursiveCharacterTextSplitter 進行評估...
Created collection:  BGE_M3_EmbeddingFunction_RecursiveCharacterTextSplitter_512_50


✅ 正在處理語料庫檔案並切割:   0%|          | 0/90 [00:00<?, ?it/s]

file_path = d:\ArtificialIntelligenceCustomerService\code\exp\chunking_evaluation\evaluation_framework\general_evaluation_data\corpora\113_2_通識教育中心_能源科技與生活[206006]_翁俊仁.html
file_name = 113_2_通識教育中心_能源科技與生活[206006]_翁俊仁.html
➡️ 處理 HTML 檔案: 113_2_通識教育中心_能源科技與生活[206006]_翁俊仁.html。使用 BeautifulSoup 提取純文字...
提取後的純文字大小（字元數）: 23032
chunks = 53 chunks created
✔️ 檔案 '113_2_通識教育中心_能源科技與生活[206006]_翁俊仁.html' 已切割成 53 個區塊，並生成 53 個中繼資料。
file_path = d:\ArtificialIntelligenceCustomerService\code\exp\chunking_evaluation\evaluation_framework\general_evaluation_data\corpora\113_2_通識教育中心_實用情境韓文[106221]_尹遠菱.html
file_name = 113_2_通識教育中心_實用情境韓文[106221]_尹遠菱.html
➡️ 處理 HTML 檔案: 113_2_通識教育中心_實用情境韓文[106221]_尹遠菱.html。使用 BeautifulSoup 提取純文字...
提取後的純文字大小（字元數）: 22847
chunks = 53 chunks created
✔️ 檔案 '113_2_通識教育中心_實用情境韓文[106221]_尹遠菱.html' 已切割成 53 個區塊，並生成 106 個中繼資料。
file_path = d:\ArtificialIntelligenceCustomerService\code\exp\chunking_evaluation\evaluation_framework\general_evaluation_data\corpora\113_2_通識教育中心_生死學[10618

✅ 正在處理語料庫檔案並切割:  18%|█▊        | 16/90 [00:00<00:00, 76.44it/s]

file_path = d:\ArtificialIntelligenceCustomerService\code\exp\chunking_evaluation\evaluation_framework\general_evaluation_data\corpora\113_2_國語文學系_中國文學史(二)[122079]_王琅.html
file_name = 113_2_國語文學系_中國文學史(二)[122079]_王琅.html
➡️ 處理 HTML 檔案: 113_2_國語文學系_中國文學史(二)[122079]_王琅.html。使用 BeautifulSoup 提取純文字...
提取後的純文字大小（字元數）: 19303
chunks = 43 chunks created
✔️ 檔案 '113_2_國語文學系_中國文學史(二)[122079]_王琅.html' 已切割成 43 個區塊，並生成 449 個中繼資料。
file_path = d:\ArtificialIntelligenceCustomerService\code\exp\chunking_evaluation\evaluation_framework\general_evaluation_data\corpora\113_2_教育學系_資料科學在教育上的應用[112173]_林娟如.html
file_name = 113_2_教育學系_資料科學在教育上的應用[112173]_林娟如.html
➡️ 處理 HTML 檔案: 113_2_教育學系_資料科學在教育上的應用[112173]_林娟如.html。使用 BeautifulSoup 提取純文字...
提取後的純文字大小（字元數）: 18043
chunks = 40 chunks created
✔️ 檔案 '113_2_教育學系_資料科學在教育上的應用[112173]_林娟如.html' 已切割成 40 個區塊，並生成 489 個中繼資料。
file_path = d:\ArtificialIntelligenceCustomerService\code\exp\chunking_evaluation\evaluation_framework\general_evaluation_data\corpora\113_2_教育學系_教學

...

生成嵌入向量中: 100%|██████████| 1/1 [00:00<00:00, 13.66it/s]


90 questions added to 'auto_questions' collection.

--- 當前 Chunking 策略結果 ---


,corpora_scores,iou_mean,iou_std,recall_mean,recall_std,precision_omega_mean,precision_omega_std,precision_mean,precision_std,chunker
0,{'chunking_evaluation/evaluation_framework/gen...,0.011404,0.007769,0.796138,0.355518,0.058879,0.028551,0.011405,0.007768,RecursiveCharacterTextSplitter_4096_250


----------------------------


✅ 所有分塊器評估完成！
=== 最終總結報告 ===


,corpora_scores,iou_mean,iou_std,recall_mean,recall_std,precision_omega_mean,precision_omega_std,precision_mean,precision_std,chunker
0,{'chunking_evaluation/evaluation_framework/gen...,0.043074,0.060915,0.384355,0.453467,0.372372,0.161638,0.043618,0.061364,RecursiveCharacterTextSplitter_512_50
1,{'chunking_evaluation/evaluation_framework/gen...,0.030957,0.051564,0.283884,0.408156,0.306022,0.147058,0.031646,0.052837,RecursiveCharacterTextSplitter_512_100
2,{'chunking_evaluation/evaluation_framework/gen...,0.033250,0.053628,0.287674,0.425635,0.329553,0.150945,0.033856,0.055142,RecursiveCharacterTextSplitter_512_150
3,{'chunking_evaluation/evaluation_framework/gen...,0.036747,0.057292,0.320985,0.436236,0.300667,0.137321,0.037489,0.058937,RecursiveCharacterTextSplitter_512_200
4,{'chunking_evaluation/evaluation_framework/gen...,0.035386,0.052737,0.308906,0.424477,0.297272,0.116579,0.036408,0.054812,RecursiveCharacterTextSplitter_512_250
5,{'chunking_evaluation/evaluation_framework/gen...,0.024753,0.029716,0.462535,0.460261,0.186973,0.090742,0.024929,0.029941,RecursiveCharacterTextSplitter_1024_50
6,{'chunking_evaluation/evaluation_framework/gen...,0.027516,0.031887,0.498842,0.467309,0.187451,0.084955,0.027639,0.031943,RecursiveCharacterTextSplitter_1024_100
7,{'chunking_evaluation/evaluation_framework/gen...,0.026700,0.031912,0.487881,0.472573,0.197366,0.077578,0.026752,0.031933,RecursiveCharacterTextSplitter_1024_150
8,{'chunking_evaluation/evaluation_framework/gen...,0.024072,0.030026,0.451299,0.459323,0.184982,0.068759,0.024184,0.030090,RecursiveCharacterTextSplitter_1024_200
9,{'chunking_evaluation/evaluation_framework/gen...,0.024916,0.030591,0.466788,0.472991,0.175776,0.078442,0.024983,0.030624,RecursiveCharacterTextSplitter_1024_250
